# Schedules Module

This notebook demonstrates the schedule utilities from `specparser.amt.schedules`.

## What is a Schedule?

A **schedule** defines the timing pattern for options strategies - specifically:
- **When to enter** a position (entry date)
- **When to exit** a position (expiry date)  
- **How much weight** to allocate to each component

## Schedule Entry Format

Each schedule consists of one or more entries in the format:

```
{entry}_{expiry}_{weight}
```

For example: `N0_F1_25` means:
- **N0**: Enter at **N**ear month, **0** days offset
- **F1**: Exit at **F**utures expiry, **1**st month
- **25**: This component gets **25%** of the weight

### Entry Codes
| Code | Meaning |
|------|---------|%
| `N0`, `N2`, `N5` | Near month + N days offset |
| `F0`, `F10`, `F15` | Far month + N days offset |

### Expiry Codes
| Code | Meaning |
|------|---------|
| `F1`, `F2`, `F3`, `F4` | Futures expiry (1st-4th month) |
| `BDa`, `BDb`, `BDc`, `BDd` | Business day schedules |
| `OVERRIDE` | Uses asset-specific override dates |
| `R2`, `R3` | Roll schedules |

In [ ]:
# Setup: Imports
import pandas as pd
from specparser.amt import (
    # Schedule loading
    get_schedule,
    find_schedules,
    # Straddle expansion
    get_expand_ym,
    find_straddle_ym,
    get_straddle_yrs,
    find_straddle_yrs,
    # Straddle parsing
    ntr, ntry, ntrm,
    xpr, xpry, xprm,
    ntrc, ntrv, xprc, xprv, wgt,
    # Days computation
    get_days_ym,
    straddle_days,
    count_straddle_days,
    count_straddles_days,
    find_straddle_days,
    # Table utilities
    table_to_rows,
    table_column,
    print_table,
    show_table,
)

# Data paths (relative to this notebook)
AMT_PATH = "../data/amt.yml"

---
## 1. Loading Schedules

### `get_schedule(path, underlying)`

Get the expiry schedule for a single asset by its Underlying value.

In [2]:
# Get schedule for a commodity asset
schedule = get_schedule(AMT_PATH, "CL Comdty")
show_table(schedule)

,schcnt,schid,asset,ntrc,ntrv,xprc,xprv,wgt
0,4,1,CL Comdty,N,0,OVERRIDE,,33.3
1,4,2,CL Comdty,N,5,OVERRIDE,,33.3
2,4,3,CL Comdty,F,10,OVERRIDE,,12.5
3,4,4,CL Comdty,F,15,OVERRIDE,,12.5


In [3]:
# Get schedule for an equity index
schedule = get_schedule(AMT_PATH, "SPX Index")
show_table(schedule)

,schcnt,schid,asset,ntrc,ntrv,xprc,xprv,wgt
0,4,1,SPX Index,N,0,F,1,25
1,4,2,SPX Index,N,0,F,2,25
2,4,3,SPX Index,N,0,F,3,25
3,4,4,SPX Index,N,0,F,4,25


In [4]:
# Get schedule for a currency asset
schedule = get_schedule(AMT_PATH, "EURUSD Curncy")
show_table(schedule)

,schcnt,schid,asset,ntrc,ntrv,xprc,xprv,wgt
0,4,1,EURUSD Curncy,N,0,BD,1,25
1,4,2,EURUSD Curncy,N,0,BD,5,25
2,4,3,EURUSD Curncy,N,0,BD,9,25
3,4,4,EURUSD Curncy,N,0,BD,13,25


**Schedule Table Columns:**

| Column | Description |
|--------|-------------|
| `schcnt` | Total number of schedule components |
| `schid` | This component's ID (1 to schcnt) |
| `asset` | The underlying asset |
| `ntrc` | Entry code (N=near, F=far) |
| `ntrv` | Entry value (day offset) |
| `xprc` | Expiry code (F1, BD, OVERRIDE, etc.) |
| `xprv` | Expiry value |
| `wgt` | Weight percentage |

In [5]:
# Asset without a schedule returns schcnt=0
no_schedule = get_schedule(AMT_PATH, "NONEXISTENT_ASSET")
print(f"Rows: {no_schedule['rows']}")

Rows: []


### `find_schedules(path, pattern, live_only)`

Find schedules for all assets matching a regex pattern.

In [6]:
# Find schedules for specific commodities (CL, GC, C)
schedules = find_schedules(AMT_PATH, pattern="^(CL|GC|C) ", live_only=True)
show_table(schedules)

,schcnt,schid,asset,ntrc,ntrv,xprc,xprv,wgt
0,4,1,C Comdty,N,0,OVERRIDE,,33.3
1,4,2,C Comdty,N,5,OVERRIDE,,33.3
2,4,3,C Comdty,F,10,OVERRIDE,,12.5
3,4,4,C Comdty,F,15,OVERRIDE,,12.5
4,4,1,CL Comdty,N,0,OVERRIDE,,33.3
5,4,2,CL Comdty,N,5,OVERRIDE,,33.3
6,4,3,CL Comdty,F,10,OVERRIDE,,12.5
7,4,4,CL Comdty,F,15,OVERRIDE,,12.5
8,4,1,GC Comdty,N,0,OVERRIDE,,33.3
9,4,2,GC Comdty,N,5,OVERRIDE,,33.3


In [7]:
# Find all live asset schedules
all_schedules = find_schedules(AMT_PATH, pattern=".", live_only=True)
print(f"Total schedule components: {len(all_schedules['rows'])}")

# Show unique asset count
assets = set(row[2] for row in all_schedules["rows"])  # col 2 is 'asset'
print(f"Unique assets with schedules: {len(assets)}")

Total schedule components: 741
Unique assets with schedules: 189


---
## 2. Understanding Schedule Components

Let's examine a schedule in detail.

In [8]:
# Get a typical commodity schedule
schedule = get_schedule(AMT_PATH, "CL Comdty")

for row in schedule["rows"]:
    schcnt, schid, asset, entry_code, entry_val, expiry_code, expiry_val, weight = row
    print(f"Component {schid}/{schcnt}:")
    print(f"  Entry: {entry_code}{entry_val} (code={entry_code}, value={entry_val})")
    print(f"  Expiry: {expiry_code}{expiry_val} (code={expiry_code}, value={expiry_val})")
    print(f"  Weight: {weight}%")
    print()

Component 1/4:
  Entry: N0 (code=N, value=0)
  Expiry: OVERRIDE (code=OVERRIDE, value=)
  Weight: 33.3%

Component 2/4:
  Entry: N5 (code=N, value=5)
  Expiry: OVERRIDE (code=OVERRIDE, value=)
  Weight: 33.3%

Component 3/4:
  Entry: F10 (code=F, value=10)
  Expiry: OVERRIDE (code=OVERRIDE, value=)
  Weight: 12.5%

Component 4/4:
  Entry: F15 (code=F, value=15)
  Expiry: OVERRIDE (code=OVERRIDE, value=)
  Weight: 12.5%



In [9]:
# Compare with a rate asset (different schedule structure)
schedule = get_schedule(AMT_PATH, "IBOXUMAE Curncy")

for row in schedule["rows"]:
    schcnt, schid, asset, entry_code, entry_val, expiry_code, expiry_val, weight = row
    print(f"Component {schid}/{schcnt}:")
    print(f"  Entry: {entry_code}{entry_val}")
    print(f"  Expiry: {expiry_code}{expiry_val}")
    print(f"  Weight: {weight}%")
    print()

Component 1/1:
  Entry: N0
  Expiry: W3
  Weight: 100%



### Interpreting Entry Codes

- **N** (Near): Entry is relative to the near (current) month
- **F** (Far): Entry is relative to the far (next) month

The value is the day offset within that month.

### Interpreting Expiry Codes

| Code | Meaning |
|------|---------|
| `F1`, `F2`, `F3`, `F4` | Exit at 1st/2nd/3rd/4th month futures expiry |
| `OVERRIDE` | Exit at asset-specific override date from CSV |
| `BD` + letter | Business day schedule (a, b, c, d variations) |
| `R2`, `R3` | Roll schedule variants |

---
## 3. Expanding Schedules to Straddles

A **straddle string** is a packed representation that includes:
- Entry year-month
- Expiry year-month  
- The entry/expiry codes and values
- The weight

### `get_expand_ym(path, underlying, year, month)`

Expand a single asset's schedule for a specific year/month.

In [10]:
# Expand CL Comdty for June 2024
straddles = get_expand_ym(AMT_PATH, "CL Comdty", 2024, 6)
show_table(straddles)

,asset,straddle
0,CL Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|
1,CL Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|
2,CL Comdty,|2024-04|2024-06|F|10|OVERRIDE||12.5|
3,CL Comdty,|2024-04|2024-06|F|15|OVERRIDE||12.5|


In [11]:
# Look at the straddle string format
for row in straddles["rows"]:
    asset, straddle = row
    print(f"Asset: {asset}")
    print(f"Straddle: {straddle}")
    print()

Asset: CL Comdty
Straddle: |2024-05|2024-06|N|0|OVERRIDE||33.3|

Asset: CL Comdty
Straddle: |2024-05|2024-06|N|5|OVERRIDE||33.3|

Asset: CL Comdty
Straddle: |2024-04|2024-06|F|10|OVERRIDE||12.5|

Asset: CL Comdty
Straddle: |2024-04|2024-06|F|15|OVERRIDE||12.5|



### `find_straddle_ym(path, year, month, pattern, live_only)`

Expand multiple assets for a specific year/month.

In [12]:
# Expand all equity assets for June 2024
equity_straddles = find_straddle_ym(AMT_PATH, 2024, 6, pattern="Index$|Equity$", live_only=True)
print(f"Found {len(equity_straddles['rows'])} straddle components")
show_table(equity_straddles)

Found 348 straddle components


,asset,straddle
0,HYG US Equity,|2024-05|2024-06|N|0|F|3|33.3|
1,HYG US Equity,|2024-05|2024-06|N|5|F|3|33.3|
2,HYG US Equity,|2024-04|2024-06|F|10|F|3|12.5|
3,HYG US Equity,|2024-04|2024-06|F|15|F|3|12.5|
4,LQD US Equity,|2024-05|2024-06|N|0|F|3|33.3|
...,...,...
343,DVN US Equity,|2024-04|2024-06|F|15|F|3|12.5|
344,LOW US Equity,|2024-05|2024-06|N|0|F|3|33.3|
345,LOW US Equity,|2024-05|2024-06|N|5|F|3|33.3|
346,LOW US Equity,|2024-04|2024-06|F|10|F|3|12.5|


In [13]:
# Expand all commodities for June 2024
commodity_straddles = find_straddle_ym(AMT_PATH, 2024, 6, pattern="Comdty$", live_only=True)
print(f"Found {len(commodity_straddles['rows'])} commodity straddle components")
show_table(commodity_straddles)

Found 72 commodity straddle components


,asset,straddle
0,LA Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|
1,LA Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|
2,LA Comdty,|2024-04|2024-06|F|10|OVERRIDE||12.5|
3,LA Comdty,|2024-04|2024-06|F|15|OVERRIDE||12.5|
4,LP Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|
...,...,...
67,FV Comdty,|2024-04|2024-06|F|15|OVERRIDE||12.5|
68,OE Comdty,|2024-05|2024-06|N|0|OVERRIDE||33.3|
69,OE Comdty,|2024-05|2024-06|N|5|OVERRIDE||33.3|
70,OE Comdty,|2024-04|2024-06|F|10|OVERRIDE||12.5|


### `get_straddle_yrs(path, underlying, start_year, end_year)`

Expand a single asset across a year range.

In [14]:
# Expand CL Comdty for all of 2024
year_straddles = get_straddle_yrs(AMT_PATH, "CL Comdty", 2024, 2024)
print(f"Total straddles: {len(year_straddles['rows'])}")
print(f"(12 months x schedule components)")

# Show first few
show_table(year_straddles)

Total straddles: 48
(12 months x schedule components)


,asset,straddle
0,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|
1,CL Comdty,|2023-12|2024-01|N|5|OVERRIDE||33.3|
2,CL Comdty,|2023-11|2024-01|F|10|OVERRIDE||12.5|
3,CL Comdty,|2023-11|2024-01|F|15|OVERRIDE||12.5|
4,CL Comdty,|2024-01|2024-02|N|0|OVERRIDE||33.3|
5,CL Comdty,|2024-01|2024-02|N|5|OVERRIDE||33.3|
6,CL Comdty,|2023-12|2024-02|F|10|OVERRIDE||12.5|
7,CL Comdty,|2023-12|2024-02|F|15|OVERRIDE||12.5|
8,CL Comdty,|2024-02|2024-03|N|0|OVERRIDE||33.3|
9,CL Comdty,|2024-02|2024-03|N|5|OVERRIDE||33.3|


### `find_straddle_yrs(path, start_year, end_year, pattern, live_only)`

Expand multiple assets across a year range.

In [15]:
# Expand all commodities for 2024
commodity_straddles = find_straddle_yrs(AMT_PATH, 2024, 2024, pattern="Comdty$", live_only=True)
print(f"Total commodity straddles for 2024: {len(commodity_straddles['rows'])}")

Total commodity straddles for 2024: 864


In [16]:
# Expand all live assets for 2024
all_straddles = find_straddle_yrs(AMT_PATH, 2024, 2024, pattern=".", live_only=True)
print(f"Total straddles for all live assets in 2024: {len(all_straddles['rows'])}")

Total straddles for all live assets in 2024: 8892


---
## 4. Straddle String Format

The straddle string packs all schedule information into a single string:

```
|2023-12|2024-01|N|0|OVERRIDE|15|33.3|
 ^^^^^^^ ^^^^^^^ ^ ^  ^^^^^^^ ^^ ^^^^
    |       |    | |     |    |   |
    |       |    | |     |    |   weight (%)
    |       |    | |     |    expiry value
    |       |    | |     expiry code
    |       |    | entry value
    |       |    entry code
    |       expiry (YYYY-MM)
    entry (YYYY-MM)
```

The format is: `|entry_date|expiry_date|ntrc|ntrv|xprc|xprv|wgt|`

In [17]:
# Example straddle string from real data
straddles = get_expand_ym(AMT_PATH, "CL Comdty", 2024, 6)
example = straddles["rows"][0][1]  # First straddle

print(f"Straddle: {example}")
print()
print("Parsed components:")
print(f"  Entry date:  {example[1:8]}")
print(f"  Expiry date: {example[9:16]}")
print(f"  Components:  {example[17:-1].split('|')}")

Straddle: |2024-05|2024-06|N|0|OVERRIDE||33.3|

Parsed components:
  Entry date:  2024-05
  Expiry date: 2024-06
  Components:  ['N', '0', 'OVERRIDE', '', '33.3']


---
## 5. Parsing Straddle Strings

The schedules module provides dedicated functions for parsing straddle strings.

### Date Extraction

In [18]:
# Get a real straddle from the data
straddles = get_expand_ym(AMT_PATH, "CL Comdty", 2024, 6)
straddle = straddles["rows"][0][1]
print(f"Straddle: {straddle}")
print()

# Entry date functions
print("Entry date parsing:")
print(f"  ntr(s)  = {ntr(straddle)!r}")    # Full entry date string
print(f"  ntry(s) = {ntry(straddle)}")      # Entry year (int)
print(f"  ntrm(s) = {ntrm(straddle)}")      # Entry month (int)
print()

# Expiry date functions
print("Expiry date parsing:")
print(f"  xpr(s)  = {xpr(straddle)!r}")    # Full expiry date string
print(f"  xpry(s) = {xpry(straddle)}")      # Expiry year (int)
print(f"  xprm(s) = {xprm(straddle)}")      # Expiry month (int)

Straddle: |2024-05|2024-06|N|0|OVERRIDE||33.3|

Entry date parsing:
  ntr(s)  = '2024-05'
  ntry(s) = 2024
  ntrm(s) = 5

Expiry date parsing:
  xpr(s)  = '2024-06'
  xpry(s) = 2024
  xprm(s) = 6


### Code and Value Extraction

In [19]:
# Continue with the same straddle
print(f"Straddle: {straddle}")
print()

# Entry code/value
print("Entry code/value:")
print(f"  ntrc(s) = {ntrc(straddle)!r}")   # Entry code
print(f"  ntrv(s) = {ntrv(straddle)!r}")   # Entry value
print()

# Expiry code/value
print("Expiry code/value:")
print(f"  xprc(s) = {xprc(straddle)!r}")   # Expiry code
print(f"  xprv(s) = {xprv(straddle)!r}")   # Expiry value
print()

# Weight
print("Weight:")
print(f"  wgt(s)  = {wgt(straddle)!r}")

Straddle: |2024-05|2024-06|N|0|OVERRIDE||33.3|

Entry code/value:
  ntrc(s) = 'N'
  ntrv(s) = '0'

Expiry code/value:
  xprc(s) = 'OVERRIDE'
  xprv(s) = ''

Weight:
  wgt(s)  = '33.3'


### Parsing Functions Summary

| Function | Returns | Example Output |
|----------|---------|----------------|
| `ntr(s)` | Entry date string | `"2023-12"` |
| `ntry(s)` | Entry year (int) | `2023` |
| `ntrm(s)` | Entry month (int) | `12` |
| `xpr(s)` | Expiry date string | `"2024-03"` |
| `xpry(s)` | Expiry year (int) | `2024` |
| `xprm(s)` | Expiry month (int) | `3` |
| `ntrc(s)` | Entry code | `"N"` |
| `ntrv(s)` | Entry value | `"5"` |
| `xprc(s)` | Expiry code | `"OVERRIDE"` |
| `xprv(s)` | Expiry value | `"15"` |
| `wgt(s)` | Weight | `"33.3"` |

---
## 6. Computing Straddle Days

### `get_days_ym(year, month)`

Get all calendar days in a month (utility function).

In [20]:
# Get all days in June 2024
days = get_days_ym(2024, 6)
print(f"June 2024 has {len(days)} days")
print(f"First day: {days[0]}")
print(f"Last day: {days[-1]}")

June 2024 has 30 days
First day: 2024-06-01
Last day: 2024-06-30


### `straddle_days(straddle)` and `count_straddle_days(straddle)`

Get or count the days spanned by a straddle.

In [21]:
# Use a real straddle from the data
straddles = get_expand_ym(AMT_PATH, "CL Comdty", 2024, 3)
straddle = straddles["rows"][0][1]
print(f"Straddle: {straddle}")

# Get all days
days = straddle_days(straddle)
print(f"Straddle spans {len(days)} days")
print(f"From: {days[0]}")
print(f"To: {days[-1]}")
print()

# Count days (faster if you just need the count)
n = count_straddle_days(straddle)
print(f"Day count: {n}")

Straddle: |2024-02|2024-03|N|0|OVERRIDE||33.3|
Straddle spans 60 days
From: 2024-02-01
To: 2024-03-31

Day count: 60


### `count_straddles_days(straddles)`

Count total days across a table of straddles.

In [22]:
# Get straddles for an asset
straddles = get_straddle_yrs(AMT_PATH, "CL Comdty", 2024, 2024)

# Total straddle-days
total_days = count_straddles_days(straddles)
print(f"CL Comdty 2024: {len(straddles['rows'])} straddles, {total_days} total straddle-days")

CL Comdty 2024: 48 straddles, 3660 total straddle-days


### `find_straddle_days(path, start_year, end_year, pattern, live_only)`

Expand straddles to a day-level table. Returns column-oriented for efficiency.

In [23]:
# Expand to day-level detail for a single asset
days_table = find_straddle_days(AMT_PATH, 2024, 2024, pattern="^CL Comdty$", live_only=False)

print(f"Orientation: {days_table['orientation']}")
print(f"Columns: {days_table['columns']}")
print(f"Total rows: {len(days_table['rows'][0])}")  # column-oriented!

# Convert to row-oriented to view
show_table(days_table)

Orientation: column
Columns: ['asset', 'straddle', 'date']
Total rows: 3660


,asset,straddle,date
0,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-01
1,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-02
2,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-03
3,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-04
4,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-05
...,...,...,...
3655,CL Comdty,|2024-10|2024-12|F|15|OVERRIDE||12.5|,2024-12-27
3656,CL Comdty,|2024-10|2024-12|F|15|OVERRIDE||12.5|,2024-12-28
3657,CL Comdty,|2024-10|2024-12|F|15|OVERRIDE||12.5|,2024-12-29
3658,CL Comdty,|2024-10|2024-12|F|15|OVERRIDE||12.5|,2024-12-30


In [24]:
# Expand multiple assets to day-level
days_table = find_straddle_days(AMT_PATH, 2024, 2024, pattern="^(CL|GC|SI) Comdty$", live_only=True)

print(f"Total day-level rows: {len(days_table['rows'][0])}")
show_table(days_table)

Total day-level rows: 10980


,asset,straddle,date
0,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-01
1,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-02
2,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-03
3,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-04
4,CL Comdty,|2023-12|2024-01|N|0|OVERRIDE||33.3|,2023-12-05
...,...,...,...
10975,SI Comdty,|2024-10|2024-12|F|15|OVERRIDE||12.5|,2024-12-27
10976,SI Comdty,|2024-10|2024-12|F|15|OVERRIDE||12.5|,2024-12-28
10977,SI Comdty,|2024-10|2024-12|F|15|OVERRIDE||12.5|,2024-12-29
10978,SI Comdty,|2024-10|2024-12|F|15|OVERRIDE||12.5|,2024-12-30


---
## 7. Practical Examples

### Example 1: Analyze Schedule Patterns by Asset Class

In [25]:
# Compare schedule complexity across asset classes
patterns = [
    ("Comdty$", "Commodities"),
    ("Index$", "Indices"),
    ("Curncy$", "Currencies"),
    ("Equity$", "Equities"),
]

for pattern, name in patterns:
    schedules = find_schedules(AMT_PATH, pattern, live_only=True)
    if schedules["rows"]:
        # Count unique assets and total components
        assets = set(row[2] for row in schedules["rows"])
        print(f"{name}: {len(assets)} assets, {len(schedules['rows'])} schedule components")
        
        # Average components per asset
        avg = len(schedules["rows"]) / len(assets) if assets else 0
        print(f"  Average components per asset: {avg:.1f}")
        print()

Commodities: 18 assets, 72 schedule components
  Average components per asset: 4.0

Indices: 10 assets, 40 schedule components
  Average components per asset: 4.0

Currencies: 75 assets, 285 schedule components
  Average components per asset: 3.8

Equities: 77 assets, 308 schedule components
  Average components per asset: 4.0



### Example 2: Monthly Straddle Distribution

In [26]:
# Get all straddles for 2024
straddles = find_straddle_yrs(AMT_PATH, 2024, 2024, pattern=".", live_only=True)

# Count straddles by expiry month
from collections import Counter
months = Counter()
for row in straddles["rows"]:
    straddle = row[1]
    month = xprm(straddle)
    months[month] += 1

print("Straddles by expiry month:")
for month in sorted(months.keys()):
    print(f"  Month {month:2d}: {months[month]:4d} straddles")

Straddles by expiry month:
  Month  1:  741 straddles
  Month  2:  741 straddles
  Month  3:  741 straddles
  Month  4:  741 straddles
  Month  5:  741 straddles
  Month  6:  741 straddles
  Month  7:  741 straddles
  Month  8:  741 straddles
  Month  9:  741 straddles
  Month 10:  741 straddles
  Month 11:  741 straddles
  Month 12:  741 straddles


### Example 3: Expiry Code Distribution

In [27]:
# Analyze expiry codes across all assets
straddles = find_straddle_yrs(AMT_PATH, 2024, 2024, pattern=".", live_only=True)

# Count by expiry code
expiry_codes = Counter()
for row in straddles["rows"]:
    straddle = row[1]
    code = xprc(straddle)
    expiry_codes[code] += 1

print("Expiry code distribution:")
for code, count in expiry_codes.most_common():
    print(f"  {code}: {count}")

Expiry code distribution:
  F: 4032
  BD: 3648
  OVERRIDE: 1056
  R: 96
  W: 60


### Example 4: Working with Day-Level Data

In [28]:
# Expand commodities to day-level
days_table = find_straddle_days(AMT_PATH, 2024, 2024, pattern="Comdty$", live_only=True)

# Convert to pandas for analysis
df = pd.DataFrame({
    "asset": days_table["rows"][0],
    "straddle": days_table["rows"][1],
    "date": days_table["rows"][2],
})

print(f"Total rows: {len(df)}")
print(f"\nAssets: {df['asset'].nunique()}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")

# Positions by date (how many straddles are active on each day)
positions_per_day = df.groupby("date").size()
print(f"\nAverage active positions per day: {positions_per_day.mean():.1f}")

Total rows: 65880

Assets: 18
Date range: 2023-11-01 to 2024-12-31

Average active positions per day: 154.3


---
## 8. Summary

### Schedule Loading Functions

| Function | Description |
|----------|-------------|
| `get_schedule(path, underlying)` | Get schedule for one asset |
| `find_schedules(path, pattern, live_only)` | Find schedules by regex pattern |

### Straddle Expansion Functions

| Function | Description |
|----------|-------------|
| `get_expand_ym(path, underlying, year, month)` | Single asset, single month |
| `find_straddle_ym(path, year, month, pattern, live_only)` | Multiple assets, single month |
| `get_straddle_yrs(path, underlying, start_year, end_year)` | Single asset, year range |
| `find_straddle_yrs(path, start_year, end_year, pattern, live_only)` | Multiple assets, year range |

### Straddle Parsing Functions

| Function | Returns | Description |
|----------|---------|-------------|
| `ntr(s)` | `str` | Entry date (YYYY-MM) |
| `ntry(s)` | `int` | Entry year |
| `ntrm(s)` | `int` | Entry month |
| `xpr(s)` | `str` | Expiry date (YYYY-MM) |
| `xpry(s)` | `int` | Expiry year |
| `xprm(s)` | `int` | Expiry month |
| `ntrc(s)` | `str` | Entry code |
| `ntrv(s)` | `str` | Entry value |
| `xprc(s)` | `str` | Expiry code |
| `xprv(s)` | `str` | Expiry value |
| `wgt(s)` | `str` | Weight |

### Days Computation Functions

| Function | Description |
|----------|-------------|
| `get_days_ym(year, month)` | All days in a month |
| `straddle_days(straddle)` | All days in a straddle period |
| `count_straddle_days(straddle)` | Count days in straddle |
| `count_straddles_days(table)` | Total days in straddles table |
| `find_straddle_days(path, start, end, pattern, live)` | Expand to day-level table |